In [1]:
import json
import pandas as pd
from datetime import datetime

import evaluate
import numpy as np
import torch
from datasets import load_dataset

/home/bujakowskih/uni/RNN/rnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
speech_data = load_dataset(
    "speech_commands", "v0.02"
)

In [3]:
speech_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 84848
    })
    validation: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 9982
    })
    test: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 4890
    })
})

In [4]:
speech_data = speech_data.filter(lambda x: len(x["audio"]["array"]) / 16_000 < 10)

In [5]:
speech_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 84843
    })
    validation: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 9981
    })
    test: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 4890
    })
})

In [6]:
subset_of_labels = ["up", "down", "left", "right", "on", "off", "yes", "no"]

In [7]:
speech_data = speech_data.filter(lambda x: x['file'].split("/")[0] in subset_of_labels)

In [8]:
speech_data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 24552
    })
    validation: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 2981
    })
    test: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 3261
    })
})

In [9]:
train = speech_data["train"]
validation = speech_data["validation"]
test = speech_data["test"]

In [10]:
SEED = 1
train = train.shuffle(seed=SEED)
validation = validation.shuffle(seed=SEED)
test = test.shuffle(seed=SEED)

In [11]:
def extract_fields(example):
    x = example["audio"]["array"]
    return {"label": example["label"], "array": np.pad(x, (0, 16000 - len(x)), constant_values=0)}

In [12]:
train = train.map(extract_fields)
validation = validation.map(extract_fields)
test = test.map(extract_fields)

In [13]:
train = train.map(remove_columns=["file", "audio", "speaker_id", "utterance_id"])
validation = validation.map(remove_columns=["file", "audio", "speaker_id", "utterance_id"])
test = test.map(remove_columns=["file", "audio", "speaker_id", "utterance_id"])

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cuda:3'
print(device)

cuda:3


In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
import numpy as np

from torch.autograd import Variable

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out


from datasets import load_metric

accuracy = load_metric("accuracy")

NUM_EPOCHS = 100
BATCH_SIZE = 256

torch.cuda.empty_cache()
model = LSTMModel(16_000, 64, 2, len(subset_of_labels))

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer=optimizer, T_max=NUM_EPOCHS, eta_min=0
)
best_val_loss = float("inf")

/tmp/ipykernel_1216364/1588589695.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy = load_metric("accuracy")


/home/bujakowskih/uni/RNN/rnn/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [16]:
def calculate_accuracy(preds, y):
    temp = torch.nn.functional.softmax(preds, dim=1)
    temp = torch.argmax(temp, dim=1)
    return (torch.sum(temp == y) / len(y)).item()

In [17]:
from tqdm import tqdm

results = pd.DataFrame(columns=["epoch", "train_loss", "train_accuracy", "val_loss", "val_accuracy"])

train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
for epoch in tqdm(range(1, NUM_EPOCHS+1)):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for batch in train_loader:
        x = batch["array"]
        x = torch.stack(x).to(device)
        x = x.unsqueeze(1)
        x = x.permute(2, 1, 0)
        y = batch["label"].to(device)
        y_pred = model(x.float())
        loss = criterion(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_accuracy += calculate_accuracy(y_pred, y)
    train_loss /= len(train_loader)
    train_accuracy = train_accuracy / len(train_loader)
    print(f"Epoch {epoch} train loss: {train_loss}, train accuracy: {train_accuracy}")

    model.eval()
    with torch.no_grad():
        val_loss = 0
        val_accuracy = 0
        for batch in DataLoader(validation, batch_size=BATCH_SIZE):
            x = batch["array"]
            x = torch.stack(x).to(device)
            x = x.unsqueeze(1)
            x = x.permute(2, 1, 0)
            y = batch["label"].to(device)
            y_pred = model(x.float())
            val_loss += criterion(y_pred, y).item()
            val_accuracy += calculate_accuracy(y_pred, y)
        val_loss /= len(validation)
        val_accuracy = val_accuracy / len(validation)
        print(f"Epoch {epoch} val loss: {val_loss}, val accuracy: {val_accuracy}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
        row = pd.DataFrame(
            {
                "epoch": [epoch],
                "train_loss": [loss.item()],
                "train_accuracy": [train_accuracy],
                "val_loss": [val_loss],
                "val_accuracy": [val_accuracy],
            }
        )
        results = pd.merge(results, row, how="outer")
    # break

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1 train loss: 2.0787476648887, train accuracy: 0.13816535333171487


  1%|          | 1/100 [04:34<7:33:05, 274.60s/it]

Epoch 1 val loss: 0.008367870206682525, val accuracy: 0.0005528860816376515
Epoch 2 train loss: 2.0660356308023133, train accuracy: 0.20729924350356063


  2%|▏         | 2/100 [09:06<7:25:43, 272.89s/it]

...

Epoch 98 val loss: 0.03823487065215112, val accuracy: 0.0007016343364216504
Epoch 99 train loss: 0.038398023849974074, train accuracy: 0.9892858744909366


 99%|█████████▉| 99/100 [7:25:25<04:29, 269.08s/it]

Epoch 99 val loss: 0.038275171309979476, val accuracy: 0.0006954000974733212
Epoch 100 train loss: 0.011081159510164676, train accuracy: 0.9982503255208334


100%|██████████| 100/100 [7:29:55<00:00, 269.95s/it]

Epoch 100 val loss: 0.03846991122148854, val accuracy: 0.0007016343364216504


In [18]:
results

,epoch,train_loss,train_accuracy,val_loss,val_accuracy
0,1,2.075794,0.138165,0.008368,0.000553
1,2,2.048750,0.207299,0.008363,0.000594
2,3,1.868140,0.311392,0.008440,0.000608
3,4,1.539145,0.461559,0.008769,0.000595
4,5,1.368787,0.524557,0.009320,0.000660
...,...,...,...,...,...
95,96,0.077500,0.996113,0.038715,0.000665
96,97,0.123601,0.957409,0.038720,0.000653
97,98,0.067961,0.984992,0.038235,0.000702
98,99,0.036201,0.989286,0.038275,0.000695


In [23]:
results['val_accuracy'].max()

0.0007227354653926953

In [20]:
results.to_csv('lstm_only_selected.csv')

In [19]:
loaded_model = LSTMModel(16_000, 64, 2, len(subset_of_labels))
loaded_model.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu')))

loaded_model.eval()

with torch.no_grad():
    val_loss = 0
    val_accuracy = 0
    for batch in DataLoader(validation, batch_size=BATCH_SIZE):
        x = batch["array"]
        x = torch.stack(x).to(device)
        x = x.unsqueeze(1)
        x = x.permute(2, 1, 0)
        y = batch["label"].to(device)
        y_pred = model(x.float())
        val_loss += criterion(y_pred, y).item()
        val_accuracy += calculate_accuracy(y_pred, y)
    val_loss /= len(validation)
    val_accuracy = val_accuracy / len(validation)
print(val_loss)
print(val_accuracy)

0.03846991122148854
0.0007016343364216504
